In [4]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [5]:
# PDFドキュメントの読込
documents = SimpleDirectoryReader('./data/pdf').load_data()

In [6]:
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Settings
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=256,
    chunk_overlap=16,
    tokenizer=tiktoken.encoding_for_model(MODEL_NAME).encode)

# 言語モデルの指定
llm = OpenAI(model=MODEL_NAME, temperature=0.3)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

In [7]:
index.storage_context.persist("./storage01")

In [10]:
sys_prompt_str = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

qa_prompt_str = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答："""

refine_prompt_str = """
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

In [11]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# テキストQAテンプレートの作成
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# リファインテンプレートの作成
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [12]:
# Chat Engineの作成
chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

In [14]:
# 質問：1回目
response = chat_engine.stream_chat("公共交通機関の交通費の上限は？")

for token in response.response_gen:
    print(token, end="")

# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線

公共交通機関の交通費の上限は、月額3万円です。ファイル名： 02賃金規則.pdf
関連度スコア: 0.8695205732268894
テキスト：
通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.8159271627288646
テキスト：
住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.7937389731729824
テキスト：
社会保険料や所得税、住⺠税など、法令で定められた控除が毎⽉の給与から差し引
かれます。
控除内容について質問がある場合は、給与明細を確認し、不明点があれば⼈事部へ
相談してください。
6. 退職⾦
1. 退職⾦の⽀給条件
みらいテクノロジー株式会社では、勤続 3 年以上の従業員に対して退職⾦を⽀給し
ます。
退職⾦は、退職時に⼀時⾦として⽀給されます。
2.
--------------------------------------------------


In [15]:
# 質問：2回目
response = chat_engine.stream_chat("交通費以外の手当にはどのようなものがありますか？")

for token in response.response_gen:
    print(token, end="")

# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線

交通費以外の手当には、以下のようなものがあります：

1. **住宅手当** - 通勤に1時間以上かかる場合に支給されます。
2. **家族手当** - 扶養家族がいる従業員に対して支給されます。
3. **退職金** - 勤続3年以上の従業員に対して支給されます。ファイル名： 02賃金規則.pdf
関連度スコア: 0.8383096341116284
テキスト：
通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.8056760424639162
テキスト：
住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.787196868275051
テキスト：
社会保険料や所得税、住⺠税など、法令で定められた控除が毎⽉の給与から差し引
かれます。
控除内容について質問がある場合は、給与明細を確認し、不明点があれば⼈事部へ
相談してください。
6. 退職⾦
1. 退職⾦の⽀給条件
みらいテクノロジー株式会社では、勤続 3 年以上の従業員に対して退職⾦を⽀給し
ます。
退職⾦は、退職時に⼀時⾦として⽀給されます。
2.
--------------------------------------------------
